<a href="https://colab.research.google.com/github/imjoung/DataScience/blob/main/Analysis/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_TOP100_%ED%81%AC%EB%A1%A4%EB%A7%81_%26_%EC%98%81%ED%99%94_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

라이브러리 임포트

In [ ]:
# 링크 수집을 위한 라이브러리 설치
!pip install selenium

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:15 http://security.ubuntu.com/ub

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
import time
import re
import pandas as pd

### 분석할 데이터 모으기(크롤링)

네이버 영화 TOP100 페이지 별 링크 수집

In [ ]:
#https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_R&page= 실시간
#https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_D&page= 일간
#https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_W&page= 주간
#https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_M&page= 월간

def getPageLinksWantRange(startPageNo, lastPageNo) :
  links = [] # 100개의 영화 링크를 수집할 리스트 변수 선언

  for pageNo in range(startPageNo -1, lastPageNo) :
    # 실시간 TOP 100 영화 링크 수집
    url = "https://serieson.naver.com/movie/top100List.nhn?rankingTypeCode=PC_D&page=" + str(pageNo+1)
    
    req = requests.get(url) # 페이지 접속
    soup = BeautifulSoup(req.text, 'lxml') # html 파싱을 위한 객체 생성
    #<div class="lst_thum_wrap"> -> <ul> -> <li> -> <a href ="">
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') # 뒤 링크만 수집하는 단계. 앞 라인도 추가 필요


    for movielink in movielinks :
      link = str(movielink.get('href'))
      # 접속할 수 있는 전체 링크 형태로 변환
      links.append("https://serieson.naver.com/" + link)
  return links

네이버 영화 제목, 평점, 장르, 줄거리 크롤링

In [ ]:
def getMovieDataFromNaverSeries(links) :
  title_infos = [] # 제목
  content_infos = [] # 줄거리
  genre_infos = [] # 장르
  score_infos = [] # 평점
  date_infos = [] # 개봉일

  url2 = "https://www.naver.com"

  driver = webdriver.Chrome('chromedriver', options=options)
  driver.get(url2)
  time.sleep(3.0) #3초 sleep

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + 't') # 새 탭 생성


  for link in links :
    print(link + '수집 중 = = 3 ')
    driver.switch_to.window(driver.window_handles[-1]) # 새탭 주소창 활성화
    time.sleep(0.1)
    driver.get(link)
    time.sleep(0.1)
    driver.switch_to.window(driver.window_handles[0]) # 새탭 주소창 활성화
    time.sleep(3.0)

    html_source = driver.page_source # req.text와 같은 동작
    html_soup = BeautifulSoup(html_source, 'lxml')

    #청소년 관람불가 영화 크롤링 할 경우 -> 인증 -> 수집 제외
    flag = html_soup.text[0:10]

    newflag = ''.join(flag)
    newflag = newflag.replace('\n', '')

    if newflag == '네이버' :
      time.sleep(1.0)

      #평점 수집
      score = driver.find_element_by_css_selector('div.score_area > em')
      score = float(score.text)
      score = int(score) #정수형으로 변경
      score_infos.append(score)
      print(score)

      
      #제목, 줄거리,장르

      try :
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(7) > a').get_attribute('href') # 7번째 li에 있는 링크 가져오기
        #장르 수집
        genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent')
        genre = genre.replace('장르','')
        genre = genre.replace('\n','')
        genre = genre.replace('\t','')
        genre_infos.append(genre)

      #print(genre)
      
      except :
        movieInfoUrl = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(6) > a').get_attribute('href') # 없으면 그냥 6번째로 간다
        #장르 수집 -> 장르 정보가 없어서, 6번째 except -> 공백으로 수집
        genre_infos.append('')      
      
      #영화 싱세보기 페이지로 이동
      movie_req = requests.get(movieInfoUrl)

      #제목 수집
      movie_soup = BeautifulSoup(movie_req.text, 'lxml')
      title = movie_soup.head.find("meta",{"property":"og:title"}).get('content')
      title_infos.append(title)
      #print(title)
      
      #줄거리 수집
      contents_texts = movie_soup.select('div.story_area > p.con_tx') # f12 페이지 소스에서 확인!
      if len(contents_texts) == 0:
        content_infos.append("줄거리 오류")

      else :
        for contents in contents_texts:
          temp = contents.text
          temp = temp.replace('\r','')
          temp = temp.replace('\xa0','')
          content_infos.append(temp)

    elif newflag == '':
        print('청불 영화로 데이터 수집하지 않습니다.')
        
  print('수집 완료 합니다 - - ! ')
  print(len(score_infos),len(genre_infos),len(content_infos))
  driver.close()

  movie_dic = {"제목" : title_infos, "평점" : score_infos, "장르" : genre_infos, "줄거리" : content_infos}
  movie_df = pd.DataFrame(movie_dic)

  #중복 데이터 삭제 (구매/대여)
  movie_df2 = movie_df.drop_duplicates("줄거리", keep='first') # 줄거리 비교하여, 중복된 영화를 삭제 (첫번째 수집 영화만 남김)
  return movie_df2


csv파일로 저장

In [ ]:
def dftoCsv(movie_df, num) :
  try :
    movie_df.to_csv('movie_my_data_'+str(num)+'.csv', 
                    sep=',', na_rep='NaN', encoding='euc-kr')
    
  except :
    print('Error')

### 크롤링 실행

In [ ]:
# 1) 크롤링 할 링크 수집

links = getPageLinksWantRange(1,5)

# 2) 영화 상세 정보 수집
movie_df2 = getMovieDataFromNaverSeries(links)

# 3) csv 파일 저장
dftoCsv(movie_df2, len(movie_df2))

https://serieson.naver.com//movie/detail.nhn?productNo=6291892수집 중 = = 3 
9
https://serieson.naver.com//movie/detail.nhn?productNo=3400212수집 중 = = 3 
7
https://serieson.naver.com//movie/detail.nhn?productNo=3024534수집 중 = = 3 
8
https://serieson.naver.com//movie/detail.nhn?productNo=2822635수집 중 = = 3 
청불 영화로 데이터 수집하지 않습니다.
https://serieson.naver.com//movie/detail.nhn?productNo=6315371수집 중 = = 3 
9
https://serieson.naver.com//movie/detail.nhn?productNo=3639460수집 중 = = 3 
청불 영화로 데이터 수집하지 않습니다.
https://serieson.naver.com//movie/detail.nhn?productNo=6023769수집 중 = = 3 
9
https://serieson.naver.com//movie/detail.nhn?productNo=6283368수집 중 = = 3 
6
https://serieson.naver.com//movie/detail.nhn?productNo=6243813수집 중 = = 3 
9
https://serieson.naver.com//movie/detail.nhn?productNo=6119777수집 중 = = 3 
7
https://serieson.naver.com//movie/detail.nhn?productNo=6139375수집 중 = = 3 
9
https://serieson.naver.com//movie/detail.nhn?productNo=2686058수집 중 = = 3 
청불 영화로 데이터 수집하지 않습니다.
https://serieson.naver.com//